In [36]:
#% pip install openpyxl
import pandas as pd
from _utilities import prepare_data, prepare_database
import configparser
import sys
sys.path.append("../tools/")
import _db_tools as db
####Input values####
tolerance = 0.025
lo_tolerance = 0.02
hi_tolerance = 0.03

def limits_gen(means, *argv):
    HI_LIMIT = []; LO_LIMIT = [] #Preallocation
    if len(argv) == 1: 
        l_tolerance = argv[0]; h_tolerance = argv[0]
    elif len(argv) == 2:
        l_tolerance = argv[0]; h_tolerance = argv[1]
    for mean in means:
        high_limit = mean + l_tolerance
        HI_LIMIT.append(round(high_limit, 4))
        low_limit = mean - h_tolerance
        LO_LIMIT.append(round(low_limit, 4))
    limits = pd.DataFrame([LO_LIMIT, HI_LIMIT]).transpose()
    limits.columns = ["LO_LIMIT", "HI_LIMIT"]
    print("Personalized limits with the given tolerance:")
    return limits

In [37]:
#Data preparation
df = prepare_data("../data/target.xlsx") #Load the output from RyR_Generator into a df
prepare_database(df, "PASSAT_B9_TOP") #Store a df inside the database of the project
dbh = db.SQLite_Data_Extractor("database.db") #Connect to the database
df = dbh.retrieve("PASSAT_B9_TOP") #Get the desired tooling data 

../database/database.db found.
Dataframe stored as *PASSAT_B9_TOP*
../database/database.db actual contents:
    measures
    table_name
    PASSAT_B9_TOP
Closed connection to: ../database/database.db
../database/database.db found.
Table PASSAT_B9_TOP retrieved succesfully.


In [38]:
resume = df.transpose().describe() #Transpose the df first due to describe() working in columns.
rough_means = list(resume.iloc[1, :].values)
means = []
for mean in rough_means: #Iterates and rounds every mean value
    mean = round(mean, 4)
    means.append(mean)
print("mean per fiber") 
print(means)

mean per fiber
[0.324, 0.3541, 0.333, 0.3572, 0.3452, 0.3707, 0.3247, 0.3519, 0.3322, 0.3586, 0.3365, 0.3656, 0.3248, 0.3538, 0.3302, 0.3588, 0.3435, 0.3688, 0.3324, 0.3593, 0.3385, 0.3623, 0.3512, 0.3728]


In [39]:
#Generates the limits and stores them in a dataframe
limits = limits_gen(means, tolerance)
#Alternative with different high and low tolerances: 
    #limits = limits_gen(means, lo_tolerance, hi_tolerance)
limits

Personalized limits with the given tolerance:


,LO_LIMIT,HI_LIMIT
0,0.2990,0.3490
1,0.3291,0.3791
2,0.3080,0.3580
3,0.3322,0.3822
4,0.3202,0.3702
5,0.3457,0.3957
6,0.2997,0.3497
7,0.3269,0.3769
8,0.3072,0.3572
9,0.3336,0.3836


In [40]:
##Creation of the .ini
#Create a ConfigParser instance and read the .ini file
class CaseSensitiveConfigParser(configparser.ConfigParser):
    '''A custom class to override optionxform and avoid uppercases being converted
    It just works F76 F76 F76 F76 F76'''
    def optionxform(self, optionstr):
        return optionstr
config = CaseSensitiveConfigParser()
config.read('../data/template.ini')
keys_list = []
for section_name in config.sections(): #Gets a keys list with the correct uppercased keys
    section = config[section_name]
    keys_list.extend(section.keys())
#Iterate through the sections and options in the .ini file
HI_LIMIT = limits.iloc[:, 1]
LO_LIMIT = limits.iloc[:, 0]
for section in config.sections():
    keys_list = list(config[section].keys())
    j = 0
    for i in range(0, len(keys_list), 2):
        key1 = keys_list[i]
        key2 = keys_list[i + 1]
        col1 = str(limits.iloc[j, 1])
        col2 = str(limits.iloc[j, 0])
        j += 1
        config[section][key1] = col1
        config[section][key2] = col2
#Loop through the sections and options in the .ini file and print them
for section in config.sections():
    print(f"[{section}]")
    for key, value in config.items(section):
        print(f"{key} = {value}")
#Save the modified data to a new .ini file
with open('../output/limits.ini', 'w') as configfile:
    for section in config.sections():
        configfile.write(f"[{section}]\n")
        keys = keys_list
        for i, key in enumerate(keys):
            configfile.write(f"{key} = {config[section][key]}\n")
            if (i + 1) % 4 == 0 and i < len(keys) - 1: #Insert a blank line every four keys
                configfile.write("\n")

[LIMITS_GENERIC]
Guia_Luz_Blanco_FB1_X_MAX = 0.349
Guia_Luz_Blanco_FB1_X_MIN = 0.299
Guia_Luz_Blanco_FB1_Y_MAX = 0.3791
Guia_Luz_Blanco_FB1_Y_MIN = 0.3291
Guia_Luz_Blanco_FB2_X_MAX = 0.358
Guia_Luz_Blanco_FB2_X_MIN = 0.308
Guia_Luz_Blanco_FB2_Y_MAX = 0.3822
Guia_Luz_Blanco_FB2_Y_MIN = 0.3322
Guia_Luz_Blanco_FB3_X_MAX = 0.3702
Guia_Luz_Blanco_FB3_X_MIN = 0.3202
Guia_Luz_Blanco_FB3_Y_MAX = 0.3957
Guia_Luz_Blanco_FB3_Y_MIN = 0.3457
Guia_Luz_Blanco_FB4_X_MAX = 0.3497
Guia_Luz_Blanco_FB4_X_MIN = 0.2997
Guia_Luz_Blanco_FB4_Y_MAX = 0.3769
Guia_Luz_Blanco_FB4_Y_MIN = 0.3269
Guia_Luz_Blanco_FB5_X_MAX = 0.3572
Guia_Luz_Blanco_FB5_X_MIN = 0.3072
Guia_Luz_Blanco_FB5_Y_MAX = 0.3836
Guia_Luz_Blanco_FB5_Y_MIN = 0.3336
Guia_Luz_Blanco_FB6_X_MAX = 0.3615
Guia_Luz_Blanco_FB6_X_MIN = 0.3115
Guia_Luz_Blanco_FB6_Y_MAX = 0.3906
Guia_Luz_Blanco_FB6_Y_MIN = 0.3406
Guia_Luz_Blanco_FB7_X_MAX = 0.3498
Guia_Luz_Blanco_FB7_X_MIN = 0.2998
Guia_Luz_Blanco_FB7_Y_MAX = 0.3788
Guia_Luz_Blanco_FB7_Y_MIN = 0.3288
Guia_Lu